# Model 16 Straight Net

This is the low baseline control.  A convolutional network that starts from the input and ends at the output.  No downsampling!


## Imports and Constants, etc.

In [ ]:
import datetime
import importlib
import keras
from keras.layers import (Dense, SimpleRNN, Input, Conv1D, 
                          LSTM, GRU, AveragePooling3D, MaxPooling3D, GlobalMaxPooling3D,
                          Conv3D, UpSampling3D, BatchNormalization, Concatenate, Add,
                          GaussianNoise, Dropout
                         )
from keras.models import Model
import nibabel as nib
import numpy as np
import pandas as pd
from pathlib import Path
import pickle
import projd
import random
import re
import scipy
import shutil
import SimpleITK # xvertseg MetaImage files
import sys
from sklearn.model_selection import train_test_split
import uuid

import matplotlib.pyplot as plt # data viz
import seaborn as sns # data viz

import imageio # display animated volumes
from IPython.display import Image # display animated volumes

from IPython.display import SVG # visualize model
from keras.utils.vis_utils import model_to_dot # visualize model

# for importing local code
src_dir = str(Path(projd.cwd_token_dir('notebooks')) / 'src') # $PROJECT_ROOT/src
if src_dir not in sys.path:
    sys.path.append(src_dir)

import util
import preprocessing
import datagen
import modelutil
import xvertseg
import augmentation

MODEL_NAME = 'model_16'

DATA_DIR = Path('/data2').expanduser()
# DATA_DIR = Path('~/data/2018').expanduser()
# UVMMC
NORMAL_SCANS_DIR = DATA_DIR / 'uvmmc/nifti_normals'
PROJECT_DATA_DIR = DATA_DIR / 'uvm_deep_learning_project'
PP_IMG_DIR = PROJECT_DATA_DIR / 'uvmmc' / 'preprocessed' # preprocessed scans dir
PP_MD_PATH = PROJECT_DATA_DIR / 'uvmmc' / 'preprocessed_metadata.pkl'
# xVertSeg
XVERTSEG_DIR = DATA_DIR / 'xVertSeg.v1'
PP_XVERTSEG_DIR = PROJECT_DATA_DIR / 'xVertSeg.v1' / 'preprocessed' # preprocessed scans dir
PP_XVERTSEG_MD_PATH = PROJECT_DATA_DIR / 'xVertSeg.v1' / 'preprocessed_metadata.pkl'


MODELS_DIR = PROJECT_DATA_DIR / 'models'
LOG_DIR = PROJECT_DATA_DIR / 'log'
TENSORBOARD_DIR = PROJECT_DATA_DIR / 'tensorboard'
TMP_DIR = DATA_DIR / 'tmp'

for d in [DATA_DIR, NORMAL_SCANS_DIR, PROJECT_DATA_DIR, PP_IMG_DIR, MODELS_DIR, LOG_DIR, 
          TENSORBOARD_DIR, TMP_DIR, PP_MD_PATH.parent, PP_XVERTSEG_DIR, PP_XVERTSEG_MD_PATH.parent]:
    if not d.exists():
        d.mkdir(parents=True)
        
%matplotlib inline
sns.set()

# I love u autoreload!
%load_ext autoreload
%autoreload 2

## Build Model


In [ ]:
def build_model(input_shape, n_a=4, n_l=8, dropout=None, noise=None):
    '''
    3D convolutional straight convolutional segmenter.
    
    dropout: proportion of activation of input to drop out. 0.0 to 1.0
    noise: std dev of noise added to input activation.
    returns: model
    '''

    x_input = Input(shape=input_shape)
    x = x_input
    
    # noise regularization
    if noise: 
        x = GaussianNoise(stddev=noise)(x)

    for i in range(n_l):
        x = Conv3D(n_a, kernel_size=(5, 5, 5), padding='same', activation='relu')(x)

    y = Conv3D(1, kernel_size=(1, 1, 1), activation='sigmoid')(x)
    
    model = Model(inputs=x_input, outputs=y)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

## Hyperparameters

In [ ]:
BATCH_SIZE = 1
MAX_QUEUE_SIZE = 10
EPOCHS = 100

SEED = 25 # random seed

PATCH_SHAPE = (32, 32, 32)
# PATCH_SHAPE = (64, 64, 64)
PATCH_SHAPE = (128, 128, 128) # good for visualization.
VALIDATION_SPLIT=0.2 # 3 samples for validation
TEST_SPLIT=0.134 # 2 samples for test
FLIP = 0.5
TRANSPOSE = True
GRAY_STD = 0.01

# Visualize model using the first set of hyperparams
N_A=4 # number of channels
N_L=0 # number of repeated layers
DROPOUT=None
NOISE=0.0015


## Data Generation

In [ ]:
infos_func = lambda: xvertseg.read_xvertseg_metadata(PP_XVERTSEG_MD_PATH)
train_gen, val_gen, test_gen = xvertseg.get_xvertseg_datagens(
    infos_func, validation_split=VALIDATION_SPLIT, test_split=TEST_SPLIT)

train_gen.config(batch_size=BATCH_SIZE, crop_shape=PATCH_SHAPE, flip=FLIP, 
                 transpose=TRANSPOSE, gray_std=GRAY_STD)
val_gen.config(batch_size=BATCH_SIZE, crop_shape=PATCH_SHAPE, flip=FLIP, 
               transpose=TRANSPOSE, gray_std=GRAY_STD)
test_gen.config(batch_size=BATCH_SIZE, crop_shape=PATCH_SHAPE, flip=FLIP, 
                transpose=TRANSPOSE, gray_std=GRAY_STD)

In [ ]:
model = build_model(input_shape=PATCH_SHAPE + (1,), n_a=N_A, n_l=N_L, 
                    dropout=DROPOUT, noise=NOISE)
print(model.summary())
SVG(model_to_dot(model).create(prog='dot', format='svg'))

In [ ]:
callbacks = [modelutil.get_tensorboard_callback(TENSORBOARD_DIR, MODEL_NAME),
             modelutil.get_logger_callback(LOG_DIR, MODEL_NAME),
             modelutil.get_checkpoint_callback(MODELS_DIR, MODEL_NAME),
            ]
history = model.fit_generator(train_gen, epochs=EPOCHS, validation_data=val_gen, 
                              callbacks=callbacks, max_queue_size=MAX_QUEUE_SIZE, 
                              use_multiprocessing=False)


### The classes need to be balanced

In [ ]:
model = build_model(input_shape=PATCH_SHAPE + (1,), n_a=N_A, n_l=N_L, 
                    dropout=DROPOUT, noise=NOISE)
# CLASS_WEIGHT = {0: 1, 1: 10000} # Does not work in 3d images.
EPOCHS=20
history = model.fit_generator(train_gen, epochs=EPOCHS, validation_data=val_gen, 
                              callbacks=callbacks, max_queue_size=MAX_QUEUE_SIZE, 
                              use_multiprocessing=False, 
#                              class_weight=CLASS_WEIGHT,
                             )
